In [1]:
import numpy as np
from datascience import *

In [2]:
# load data
mc = Table().read_table('../minilab6/modechoice.csv')
mc

observation_id,choice,availability_drive_alone,availability_shared_ride_2,availability_shared_ride_3+,availability_walk_transit_walk,availability_drive_transit_walk,availability_walk_transit_drive,availability_walk,availability_bicycle,household_id,person_id,tour_id,tour_origin_taz,primary_dest_taz,age,household_size,household_income,household_income_values,transit_subsidy,transit_subsidy_amount,cross_bay,total_travel_time_drive_alone,total_travel_time_shared_ride_2,total_travel_time_shared_ride_3+,total_travel_time_walk_transit_walk,total_travel_time_drive_transit_walk,total_travel_time_walk_transit_drive,total_travel_time_walk,total_travel_time_bicycle,total_travel_distance_drive_alone,total_travel_distance_shared_ride_2,total_travel_distance_shared_ride_3+,total_travel_distance_walk,total_travel_distance_bicycle,total_travel_cost_drive_alone,total_travel_cost_shared_ride_2,total_travel_cost_shared_ride_3+,total_travel_cost_walk_transit_walk,total_travel_cost_drive_transit_walk,total_travel_cost_walk_transit_drive,age_ctgry
1,drive_alone,1,1,1,1,1,1,1,1,1043613,1,0,1004,969,53,2,6,87500,0,-1,0,20.97,20.97,20.97,83.56,63.05,67.29,167.6,41.9,9.1,9.1,9.1,8.38,8.38,1.6817,0.9609,0.6726,5.84,3.22,3.02,45-64
2,drive_alone,1,1,1,1,1,1,1,1,1043613,1,1,1004,969,53,2,6,87500,0,-1,0,22.16,22.16,22.16,87.38,77.79,58.01,167.6,41.9,9.1,9.1,9.1,8.38,8.38,1.6817,0.9609,0.6726,4.83,3.22,3.22,45-64
3,shared_ride_2,1,1,1,1,1,1,0,0,1045607,1,0,908,116,57,1,4,42500,0,-1,1,47.98,47.98,47.98,140.63,131.49,137.39,nan,nan,31.94,31.94,31.94,nan,nan,9.1125,5.2071,3.645,6.95,7.75,5.98,45-64
4,bike,0,1,1,1,0,1,1,1,1049908,1,0,30,12,50,1,2,17500,0,-1,0,nan,15.05,15.05,26.46,nan,16.77,42.8,10.7,nan,2.23,2.23,2.14,2.14,nan,0.2355,0.1649,3.22,nan,3.22,45-64
5,walk,0,1,1,1,0,1,1,1,1049908,1,1,30,9,50,1,2,17500,0,-1,0,nan,11.72,11.72,43.86,nan,28.89,19.2,4.8,nan,0.96,0.96,0.96,0.96,nan,0.1014,0.071,3.22,nan,3.22,45-64
6,shared_ride_2,1,1,1,1,1,1,1,1,1051388,2,0,89,4,48,3,7,125000,0,-1,0,26.32,26.32,26.32,42.1,55.48,58.48,143.2,35.8,7.4,7.4,7.4,7.16,7.16,1.3675,0.7815,0.547,3.22,3.98,3.22,45-64
7,drive_alone,1,1,1,1,1,1,1,1,1051388,2,1,89,23,48,3,7,125000,0,-1,0,32.07,32.07,32.07,86.89,91.7,83.35,184.8,46.2,9.47,9.47,9.47,9.24,9.24,1.7501,1,0.7,3.22,3.22,3.22,45-64
8,shared_ride_3,1,1,1,1,1,1,1,1,1051388,3,0,89,71,10,3,7,125000,0,-1,0,16.58,16.58,16.58,53.32,46.76,46.77,94,23.5,4.7,4.7,4.7,4.7,4.7,0.8686,0.4964,0.3474,3.22,3.22,3.22,05-19
9,walk_transit_walk,1,1,1,1,1,1,1,0,1069723,1,0,86,12,50,2,5,62500,0,-1,0,21.93,21.93,21.93,54.13,42.72,44.78,95.6,nan,4.87,4.87,4.87,4.78,nan,0.9,0.5143,0.36,3.22,3.22,3.22,45-64
10,walk,1,1,1,1,1,1,1,1,1093539,1,0,90,111,56,3,8,175000,1,999998,0,33.13,33.13,33.13,100.65,100.39,100.39,192.4,48.1,9.86,9.86,9.86,9.62,9.62,1.8222,1.0412,0.7288,3.22,3.22,3.22,45-64


In [3]:
# get appropriate rows:
drive_transit_av = mc.where('availability_drive_alone',1).where('availability_walk_transit_walk',1)
drive_or_transit = drive_transit_av.where(np.in1d(drive_transit_av['choice'],['drive_alone','walk_transit_walk']))

#calculate VMT:
vmt = np.sum(drive_or_transit.where('choice','drive_alone').column('total_travel_distance_drive_alone'))
print ('Total drive_alone VMT = %f'%vmt)

Total drive_alone VMT = 2092.150000


In [4]:
# kNN
def distance(x, y):
    return np.sqrt(np.sum((x-y)**2,1))

class KNearestNeighbors():
    def __init__ (self, n_neighbors=5):
        '''
        n_neighbors: number of neighbors
        '''
        self.n_neighbors = n_neighbors
    
    def fit(self, input_data, response):
        '''
        input_data: a table, the values of this table will be used to 
            compute the distance to the neighbors
        response: a table with one column, the values in this column 
            represent the category of the thing we are trying to predict 
        '''
        self.input_data = input_data.values
        self.response = response.values.flatten()
    
    def predict(self, to_predict, return_kneighbor_inds=True):
        '''
        to_predict: A single input data point. It should contain one value 
           for each of the columns in the input_data table.
        return_kneighbor_inds: boolean. If True, return the indices of the
            nearest neighbors from the input table, otherwise, only the 
            majority category of the k-nearest neighbors is returned.
        '''
         #get distance input_data to predict"
        dists = distance(to_predict, self.input_data)

        #get indices of k nearest points
        inds = np.argsort(dists)[0:self.n_neighbors]

        #return the most common response among the neighbors
        most_common_response = (np.argmax(np.bincount(self.response[inds])))
        if return_kneighbor_inds:
            return most_common_response, inds
        return most_common_response
            

In [5]:
#create kNN input:
tt_delta = (drive_or_transit.column('total_travel_time_walk_transit_walk')-
            drive_or_transit.column('total_travel_time_drive_alone'))

cost_delta = (drive_or_transit.column('total_travel_cost_walk_transit_walk')-
              drive_or_transit.column('total_travel_cost_drive_alone'))

choice_drive = np.where(drive_or_transit['choice']=='drive_alone',1,0)


input_table = Table().with_columns(['tt_delta', tt_delta,
                                    'cost_delta', cost_delta,
                                    'choice=drive',choice_drive])


In [6]:
#scale travel time to put in $:
input_table['tt_delta_$'] = 20*(input_table['tt_delta']/60.)
input_table

tt_delta,cost_delta,choice=drive,tt_delta_$
62.59,4.1583,1,20.8633
65.22,3.1483,1,21.74
54.82,1.4699,1,18.2733
32.2,2.32,0,10.7333
26.79,2.3866,1,8.93
36.39,2.4956,0,12.13
36.39,2.4956,0,12.13
43.86,1.7601,0,14.62
43.09,2.1759,0,14.3633
136.98,1.3738,0,45.66


In [7]:
#Run kNN with subsidy:
subsidy = 1
input_data = input_table.select(['tt_delta_$','cost_delta'])

# Your task: Use the input_table.select() method to select the column with the response variable
#answer key:
response = input_table.select(['choice=drive'])

# Your task: Create an instance of the KNearestNeighbors class, let's set n_neighbors to 5.
#answer key:
kNN = KNearestNeighbors(10)

# Your task: Load in the nearest neighbor candidates using the fit method:
kNN.fit(input_data, response)

drive = []
for row in input_data.rows:
    to_predict = [row[0],row[1]-subsidy]
    prediction, nn_inds = kNN.predict(to_predict, True)
    drive.append(prediction)
    predicted_travel_mode = 'drive' if prediction ==1 else 'take transit'
#     print ('Based on the k nearest neighbors, the predicted travel mode is %s' %predicted_travel_mode)

In [8]:
#calculate new VMT
print('The updated VMT after a $%.2f subsidy is %f' 
      %(subsidy, np.sum(drive_or_transit.where(drive)['total_travel_distance_drive_alone'])))

The updated VMT after a $1.00 subsidy is 1638.230000
